# Forecasting Prediction Using LightGBM Model

In [2]:
import pandas as pd
import numpy as np
import datetime
import warnings
#import lightgbm as lgb
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

## Feature Engineering

In this section we will generate additional feature categories to provide more insight for the machine learning algorithm:

* Time-Related Features
* Lag/Shifted Features
* Rolling Mean/Moving Average

### Time-Related Features

Time-Related features are features generated by the timestap of each observation.

Generated these additional time-related features can provide insights to improve your model.

For example you could use the timestamp and generate a feature called *isHoliday* which tells you whether that day is a holiday or not. From this feature you might see that sales are higher on holidays.

### Lag/Shifted Features

Lag features are useful because the value observed at time $t$ is highly dependent on the value observed at time $t-1$

### Rolling Mean/Moving Average